In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/SIH/data/raw_data"

In [4]:
def generate_synthetic_images_for_minorities(data_dir, target_count=210):
    """Generate synthetic images for minority classes using ImageDataGenerator"""

    print("Generating synthetic images for minority classes...")

    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.2,
        zoom_range=[0.8, 1.3],
        horizontal_flip=True,
        brightness_range=[0.7, 1.8],
        channel_shift_range=20,
        fill_mode='reflect'
    )

    class_counts = {}
    for breed_folder in os.listdir(data_dir):
        breed_path = os.path.join(data_dir, breed_folder)
        if os.path.isdir(breed_path):
            count = len([f for f in os.listdir(breed_path)
                        if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            class_counts[breed_folder] = count

    # Generate synthetic images for classes with < target_count images
    for breed, count in class_counts.items():
        if count < target_count:
            needed = target_count - count
            print(f"Generating {needed} synthetic images for {breed}")

            breed_path = os.path.join(data_dir, breed)

            # Get existing images
            existing_images = [f for f in os.listdir(breed_path)
                             if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

            # Generate synthetic images
            generated_count = 0
            for img_file in existing_images:
                if generated_count >= needed:
                    break

                img_path = os.path.join(breed_path, img_file)
                try:
                    # Load image
                    from tensorflow.keras.utils import load_img, img_to_array
                    img = load_img(img_path, target_size=(224, 224))
                    img_array = img_to_array(img)
                    img_array = np.expand_dims(img_array, axis=0)

                    # Generate variations
                    variations_needed = min(2, needed - generated_count)
                    i = 0
                    for batch in datagen.flow(img_array, batch_size=1): #batch_size=1: Generates one image at a time  #Creates an infinite generator that produces augmented versions of the image.
                        if i >= variations_needed:
                            break

                        # Save synthetic image
                        synthetic_img = batch[0].astype('uint8')  #batch[0]: Gets the first (and only) image from the batch
                        from PIL import Image
                        pil_img = Image.fromarray(synthetic_img)
                        synthetic_name = f"syn_{breed}_{generated_count}_{i}.jpg"
                        synthetic_path = os.path.join(breed_path, synthetic_name)
                        pil_img.save(synthetic_path)

                        generated_count += 1
                        i += 1

                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
                    continue

    print("Synthetic image generation completed!")



In [5]:
# Generate synthetic images for minority classes
generate_synthetic_images_for_minorities(DATA_DIR, target_count=210)

Generating synthetic images for minority classes...
Generating 37 synthetic images for Murrah
Generating 32 synthetic images for Kankrej Cow
Generating 108 synthetic images for Jaffrabadi Buffalo
Generating 24 synthetic images for Hallikar Cow
Generating 62 synthetic images for Rathi Cow
Generating 24 synthetic images for Nili ravi Buffalo
Synthetic image generation completed!
